In [2]:
import pandas as pd

In [3]:
# test if opening file works
df = pd.read_csv("FONTS.CSV")
count = 0

template = {"substrateCount": 0}

with open("fineTuningData.json", "w") as file:
    for _, row in df.iterrows():
        if count == 5:
            break


In [1]:
# template definitions

template = { "substrateCount":  0,
"substrates":
  [
    {
      "placement": "",
      "additionalNotes": "",
      "thisIsntReallyASign": False,
      "notASignDescription": "",  
      "typefaces":
        [
          {
            "typefaceStyle": [],
            "copy": "",
            "letteringOntology": [],
            "messageFunction": "",
            "covidRelated": False,
            "additionalNotes": ""
          }
        ],
      "confidence": -1,
      "confidenceReasoning": "",
      "additionalInfo": "",
    },
  ]
}
template = {"substrateCount": 0, "substrates": []}
typeface_template = {"typefaceStyle": [], "copy": "", "letteringOntology": [], "messageFunction": "", "covidRelated": False, "additionalNotes": ""}
substrate_template = {"placement": "", "additionalNotes": "", "thisIsntReallyASign": False, "notASignDescription": "", "typefaces": [], "confidence": -1, "confidenceReasoning": "", "additionalInfo": ""}


In [51]:
# original code, updates to it made below, kept for backwards reference
# import json
# count = 0

# placement_col_names = ["Placement (1)", "Placement (2)", "Placement", "Placement (3)"]
# with open("fineTuningData.jsonl", "w") as file:
#     for _, row in df.iterrows():
#         # if count == 5:
#         #     break
#         template = {"substrateCount": 0, "substrates": []}
#         template["substrateCount"] = row["Number of substrates"]
#         if not row["Placement (1)"]:  # there should always be at least one substrate
#             continue

#         for i in range(template["substrateCount"]):
#             substrate_template = {"placement": "", "additionalNotes": "", "thisIsntReallyASign": False, "notASignDescription": "", "typefaces": [], "confidence": -1, "confidenceReasoning": "", "additionalInfo": ""}
#             placement_col = placement_col_names[i]
#             idx = df.columns.get_loc(placement_col)
#             substrate_template["placement"] =  None if pd.isna(row.iloc[idx]) else row.iloc[idx].strip()
#             if row.iloc[idx] is None:
#                 print("Error with indexing, placement cell was empty")
#             substrate_template["additionalNotes"] = None if pd.isna(row.iloc[idx + 1]) else row.iloc[idx + 1].strip()
#             substrate_template["thisIsntReallyASign"] = False if (row.iloc[idx + 2] == "FALSE" or pd.isna(row.iloc[idx + 2]) or row.iloc[idx + 2] == "") else True
#             substrate_template["notASignDescription"] = None if pd.isna(row.iloc[idx + 3]) else row.iloc[idx + 3].strip()

#             num_typefaces = int(row.iloc[idx+4])
#             typeface_start = int(idx + 5) # shoudl be first typeface style column
#             for i in range(typeface_start, typeface_start + num_typefaces*7 - 2, 7):  # iterating over the typefaces, i is the first typefaceStyle column each time
#                 typeface_template = {"typefaceStyle": [], "copy": "", "letteringOntology": [], "messageFunction": "", "covidRelated": False, "additionalNotes": ""}
#                 typeface_template["typefaceStyle"] = row.iloc[i].strip().split(",")  # allowing for multiple fonts
#                 typeface_template["copy"] = row.iloc[i+1].strip()
#                 typeface_template["letteringOntology"] = row.iloc[i+2].strip().split(",")
#                 typeface_template["messageFunction"] = None if pd.isna(row.iloc[i+3]) else row.iloc[i+3].strip()
#                 typeface_template["covidRelated"] = False if (row.iloc[i + 4] == "FALSE" or  pd.isna(row.iloc[idx + 4]) or row.iloc[idx + 4] == "") else True
#                 typeface_template["additionalNotes"] = None if pd.isna(row.iloc[i+6]) else row.iloc[i+6]
#                 substrate_template["typefaces"].append(typeface_template)

            
#             substrate_template["confidence"] = -1 if pd.isna(row.iloc[idx + 61]) else int(row.iloc[idx+61])
#             substrate_template["confidenceReasoning"] = None if pd.isna(row.iloc[idx + 62]) else row.iloc[idx+62]
#             substrate_template["additionalInfo"] =  None if pd.isna(row.iloc[idx + 63]) else row.iloc[idx+63]
#             template["substrates"].append(substrate_template)
            
                

        
        
#         file.write(json.dumps(template) + '\n')
#         count += 1

In [4]:
import base64
from pathlib import Path

def encode_image(images_dir, image_name):
    '''
    Encodes an image into base64 so it can be stored in a jsonl file
    '''

    # find image
    root_path = Path(images_dir)
    for file in root_path.rglob(image_name):  # rglob() searches recursively
        with open(str(file), "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")

    return None


def construct_messages_object(images_dir, image_name):
    '''
    Constructs overall messages object, leaves fields blank for another function to fill in
    '''
    messages_object = {"messages": []}
    system_instructions = {
        "role": "system",
        "content": """You are an assistant that's helping me with this research project. The structure of this project is that we have had undergraduates label traits of images manually. These traits include things like ontology, substrates, font information, text, covid-relation, confidence, and more. You are now going to be doing what they have been doing. When provided with an image, you should return a json object that describes the image according to the structure given here. The structure you should return as is provided below, along with an example. Keep in mind there can be multiple substrates (up to 4), with multiple fonts for each substrate (up to 8). Many of the images will have multiple substrates, look closely in the images for any other substrates which are clear and sharp. For information on what each possible field can mean and examples of each, refer to the files you have available. When annotating the text that is in the image, ensure that the markdown guide that you have available to you is used to record the text.
    
        This is the format you should return to me in, with comments denoting the purpose of the field for some:
    
        |||{return format}
        {
            "substrateCount": // the number of substrates in this image,
            "substrates": [
                {
                    "placement": // refer to placement examples in screenshots of form,
                    "additionalNotes": // not always necessary to include,
                    "thisIsntReallyASign": // set this field to true if this doesn't really fit any of the placement categories and isn't a sign, else false,
                    "notASignDescription": // use this if the previous field was true, describe the placement,
                    "typefaces": [
                        {
                            "typefaceStyle": [], // can be multiple, choose from the options provided to you below,
                            "copy": // make sure that the text is annotated according to the markdown guide in one of the screenshots provided to you,
                            "letteringOntology": [], // refer to the OC Fonts: Codebook Descriptions & Photo Examples file for examples and descriptions of what each of these are,
                            "messageFunction": // again refer to the OC Fonts: Codebook Descriptions & Photo Examples file for examples and descriptions of what each of these are,
                            "covidRelated": // is this text covid related?,
                            "additionalNotes": // any additional notes needed about this image
                        }
                    ],
                    "confidence": // overall confidence in your annotation, 0 being the lowest, 5 the highest,
                    "confidenceReasoning": // reasoning for confidence rating,
                    "additionalInfo": // any additional info about the substrate, not always necessary to include
                }
            ]
        }
        |||
    
        This is an example of the returned product you should give to me. This example only has one substrate but other images may have multiple.
    
        |||{return example}
        {
            "substrateCount": 1,
            "substrates": [
                {
                    "placement": "Window-stuck",
                    "additionalNotes": "decal stickers on a parking meter",
                    "thisIsntReallyASign": false,
                    "notASignDescription": "",
                    "typefaces": [
                        {
                            "typefaceStyle": ["Serif", "Stylized"],
                            "copy": "Please, no food or\\ndrink in the store.\\nThank You!",
                            "letteringOntology": ["Painted", "Pan-face"],
                            "messageFunction": "Operational information",
                            "covidRelated": false,
                            "additionalNotes": "Text is center aligned, \\\"OPEN\\\" is larger than \\\"5PM DAILY\\\""
                        }
                    ],
                    "confidence": 4,
                    "confidenceReasoning": "Could be painted or pan-face or both",
                    "additionalInfo": "Image heavily cut off"
                }
            ]
        }
        |||
    
        These are some of the options for message function, typeface style, ontology and placements:
    
        |||{
            "typeface": [
                "Serif", "Sans serif", "Slab serif", "Script", "Stylized", "Quirky"
            ],
            "lettering_ontology": [
                "Printed", "Decal", "Painted", "Pan channel", "Pan face", "Handmade",
                "Embossed", "Debossed", "Pen or marker", "Reader board", "Spray paint",
                "LED", "Other electronic", "Neon", "Tile", "Chalk", "House number", "Ghost sign"
            ],
            "placements": [
                "Window-stuck", "Window-placed", "Awning/canopy", "Blade", "Fascia",
                "Marquee", "Hanging", "Name-plate", "Painted wall", "Freestanding",
                "Parapet", "Ground", "Bench", "Flag", "Pole-mounted", "Post and panel",
                "Pylon", "Banner", "Wall-placed", "Wall-stuck", "Other-stuck", "Snipe",
                "Graffiti", "Infrastructure", "Memorial", "Sticker"
            ],
            "message_function": [
                "Identification", "Address", "Joint tenant", "Operational information",
                "Advisement/regulation", "Directory", "Generic information", "Menu of options",
                "Commemoration", "Street name", "Advertisement", "Wayfinding", "Infrastructure",
                "Covid-related"
            ]
        }|||
        """
    }

    messages_object["messages"].append(system_instructions)
    base64_image = encode_image(images_dir, image_name)
    if not base64_image:
        return None
    user_message = {"role": "user", "content": [{
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
                        # "image_url": "temp bs"
                    }]}
    messages_object["messages"].append(user_message)
    assistant_message = {"role": "assistant", "content": []}
    messages_object["messages"].append(assistant_message)
    return messages_object

In [8]:
import json

def create_fine_tune(pathName: str, rowsCount=float("inf"), images_dir="images/"):
    '''
    Creates training examples for jsonl file. If it cannot find the image
    corresponding to a row in the specified images_dir, it is skipped.

    '''
    
    count = 0

    placement_col_names = ["Placement (1)", "Placement (2)", "Placement", "Placement (3)"]
    with open(pathName, "w") as file:
        for _, row in df.iterrows():
            filename = row["Photo name"].strip()
            if pd.isna(filename):
                continue
            messages_object = construct_messages_object(images_dir, filename)
            if not messages_object:
                continue

                      
            if count >= rowsCount:
                break
            template = {"substrateCount": 0, "substrates": []}
            template["substrateCount"] = row["Number of substrates"]
            if not row["Placement (1)"]:  # there should always be at least one substrate
                continue
    
            for i in range(template["substrateCount"]):
                substrate_template = {"placement": "",
                                      "additionalNotes": "",
                                      "thisIsntReallyASign": False,
                                      "notASignDescription": "",
                                      "typefaces": [],
                                      "confidence": -1,
                                      "confidenceReasoning": "",
                                      "additionalInfo": ""}
                placement_col = placement_col_names[i]
                idx = df.columns.get_loc(placement_col)
                substrate_template["placement"] =  None if pd.isna(row.iloc[idx]) else row.iloc[idx].strip()
                if row.iloc[idx] is None:
                    print("Error with indexing, placement cell was empty")
                substrate_template["additionalNotes"] = None if pd.isna(row.iloc[idx + 1]) else row.iloc[idx + 1].strip()
                substrate_template["thisIsntReallyASign"] = False if (row.iloc[idx + 2] == "FALSE" or pd.isna(row.iloc[idx + 2]) or row.iloc[idx + 2] == "") else True
                substrate_template["notASignDescription"] = None if pd.isna(row.iloc[idx + 3]) else row.iloc[idx + 3].strip()
    
                num_typefaces = int(row.iloc[idx+4])
                typeface_start = int(idx + 5) # shoudl be first typeface style column
                for i in range(typeface_start, typeface_start + num_typefaces*7 - 2, 7):  # iterating over the typefaces, i is the first typefaceStyle column each time
                    typeface_template = {"typefaceStyle": [], "copy": "", "letteringOntology": [], "messageFunction": "", "covidRelated": False, "additionalNotes": ""}
                    typeface_template["typefaceStyle"] = row.iloc[i].strip().split(",")  # allowing for multiple fonts
                    typeface_template["copy"] = row.iloc[i+1].strip()
                    typeface_template["letteringOntology"] = row.iloc[i+2].strip().split(",")
                    typeface_template["messageFunction"] = None if pd.isna(row.iloc[i+3]) else row.iloc[i+3].strip()
                    typeface_template["covidRelated"] = False if (row.iloc[i + 4] == "FALSE" or  pd.isna(row.iloc[idx + 4]) or row.iloc[idx + 4] == "") else True
                    typeface_template["additionalNotes"] = None if pd.isna(row.iloc[i+6]) else row.iloc[i+6]
                    substrate_template["typefaces"].append(typeface_template)
    
                
                substrate_template["confidence"] = -1 if pd.isna(row.iloc[idx + 61]) else int(row.iloc[idx+61])
                substrate_template["confidenceReasoning"] = None if pd.isna(row.iloc[idx + 62]) else row.iloc[idx+62]
                substrate_template["additionalInfo"] =  None if pd.isna(row.iloc[idx + 63]) else row.iloc[idx+63]
                template["substrates"].append(substrate_template)
                
                    
    
            
            messages_object["messages"][2]["content"] = json.dumps(template)  # was previously json.dumps(template)
            file.write(json.dumps(messages_object) + '\n')
            count += 1
    print("Successfully collected", count, " training examples")

create_fine_tune("file-fineTuningDataFinal.jsonl", 10)

Successfully collected 10  training examples


In [11]:
# error checking, code is from openai cookbook, pretty old, take with a grain of salt
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

data_path = "file-fineTuningDataFinal.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
# for message in dataset[0]["messages"]:
#     print(message)
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")


Num examples: 10
First example:
Found errors:
missing_content: 10
